In [ ]:
!git clone --depth=1 https://github.com/FelipeMarcelino/2048-Gym.git # work/gym-env

fatal: destination path '2048-Gym' already exists and is not an empty directory.


In [ ]:
!pip list |grep parl
!pip list |grep paddlepaddle-gpu
!pip list |grep gym

parl                   1.3.1          
paddlepaddle-gpu       1.8.0.post97   
gym                    0.12.1         


In [ ]:
!pip uninstall -y parl  
!pip uninstall -y pandas scikit-learn 
!pip install parl==1.3.1
!pip install gym

Uninstalling parl-1.3.1:
  Successfully uninstalled parl-1.3.1
Looking in indexes: https://pypi.mirrors.ustc.edu.cn/simple/
  Using cached https://mirrors.tuna.tsinghua.edu.cn/pypi/web/packages/62/79/590af38a920792c71afb73fad7583967928b4d0ba9fca76250d935c7fda8/parl-1.3.1-py2.py3-none-any.whl
Looking in indexes: https://pypi.mirrors.ustc.edu.cn/simple/


In [ ]:
#!cd 2048-Gym/gym-game2048 终端运行创建环境
#!pip install -e . 
import sys
sys.path.append('/home/aistudio/2048-Gym/gym-game2048')

In [ ]:
import parl
from parl import layers
import paddle.fluid as fluid
import copy
import numpy as np
import os
import gym
import gym_game2048
from parl.utils import logger
import collections

In [ ]:
LEARN_FREQ = 5 # 训练频率，不需要每一个step都learn，攒一些新增经验后再learn，提高效率
MEMORY_SIZE = 2e4    # replay memory的大小，越大越占用内存
MEMORY_WARMUP_SIZE = 200  # replay_memory 里需要预存一些经验数据，再从里面sample一个batch的经验让agent去learn
BATCH_SIZE = 32   # 每次给agent learn的数据数量，从replay memory随机里sample一批数据出来
GAMMA = 0.99 # reward 的衰减因子，一般取 0.9 到 0.999 不等
LEARNING_RATE = 0.001 # 学习率
MAX_EPISODE = 2000

In [ ]:
class Model(parl.Model):
    def __init__(self, act_dim):
        self.cnn1 = layers.conv2d(num_filters=256, filter_size=2, padding="SAME", stride=1,act="leaky_relu")
        self.cnn2 = layers.conv2d(num_filters=256, filter_size=2, padding="SAME", stride=1,act="leaky_relu")
        self.cnn3 = layers.conv2d(num_filters=256, filter_size=2, padding="SAME", stride=1,act="leaky_relu")
        self.cnn4 = layers.conv2d(num_filters=256, filter_size=2, padding="SAME", stride=1,act="leaky_relu")
        self.fc1 = layers.fc(size=act_dim, act="leaky_relu")
    def value(self, obs):
        # 定义网络
        # 输入state，输出所有action对应的Q，[Q(s,a1), Q(s,a2), Q(s,a3)...]
        h1 = self.cnn1(obs)
        h2 = self.cnn2(h1)
        h3 = self.cnn3(h2)
        h4 = self.cnn4(h3)
        Q = self.fc1(h4)
        return Q

In [ ]:
from parl.algorithms import DQN

In [ ]:
class Agent(parl.Agent):
    def __init__(self,
                 algorithm,
                 obs_dim,
                 act_dim,
                 e_greed=0.1,
                 e_greed_decrement=0):
        assert isinstance(act_dim, int)
        self.obs_dim = obs_dim
        self.act_dim = act_dim
        super(Agent, self).__init__(algorithm)

        self.global_step = 0
        self.update_target_steps = 200  # 每隔200个training steps再把model的参数复制到target_model中

        self.e_greed = e_greed  # 有一定概率随机选取动作，探索
        self.e_greed_decrement = e_greed_decrement  # 随着训练逐步收敛，探索的程度慢慢降低

    def build_program(self):
        self.pred_program = fluid.Program()
        self.learn_program = fluid.Program()

        with fluid.program_guard(self.pred_program):  # 搭建计算图用于 预测动作，定义输入输出变量
            obs = layers.data(
                name='obs', shape=self.obs_dim, dtype='float32')
            self.value = self.alg.predict(obs)

        with fluid.program_guard(self.learn_program):  # 搭建计算图用于 更新Q网络，定义输入输出变量
            obs = layers.data(
                name='obs', shape=self.obs_dim, dtype='float32')
            action = layers.data(name='act', shape=[1], dtype='int32')
            reward = layers.data(name='reward', shape=[], dtype='float32')
            next_obs = layers.data(
                name='next_obs', shape=self.obs_dim, dtype='float32')
            terminal = layers.data(name='terminal', shape=[], dtype='bool')
            self.cost = self.alg.learn(obs, action, reward, next_obs, terminal)

    def sample(self, obs):
        sample = np.random.rand()  # 产生0~1之间的小数
        if sample < self.e_greed:
            act = np.random.randint(self.act_dim)  # 探索：每个动作都有概率被选择
        else:
            act = self.predict(obs)  # 选择最优动作
        self.e_greed = max(0.01, self.e_greed - self.e_greed_decrement)  # 随着训练逐步收敛，探索的程度慢慢降低
        return act

    def predict(self, obs):  # 选择最优动作
        obs = np.expand_dims(obs, axis=0)
        pred_Q = self.fluid_executor.run(
            self.pred_program,
            feed={'obs': obs.astype('float32')},
            fetch_list=[self.value])[0]
        pred_Q = np.squeeze(pred_Q, axis=0)
        act = np.argmax(pred_Q)  # 选择Q最大的下标，即对应的动作
        return act

    def learn(self, obs, act, reward, next_obs, terminal):
        # 每隔200个training steps同步一次model和target_model的参数
        if self.global_step % self.update_target_steps == 0:
            self.alg.sync_target()
        self.global_step += 1

        act = np.expand_dims(act, -1)
        feed = {
            'obs': obs.astype('float32'),
            'act': act.astype('int32'),
            'reward': reward,
            'next_obs': next_obs.astype('float32'),
            'terminal': terminal
        }
        cost = self.fluid_executor.run(
            self.learn_program, feed=feed, fetch_list=[self.cost])[0]  # 训练一次网络
        return cost

In [ ]:
import random
class ReplayMemory(object):
    def __init__(self, max_size):
        self.buffer = collections.deque(maxlen=max_size)

    # 增加一条经验到经验池中
    def append(self, exp):
        self.buffer.append(exp)

    # 从经验池中选取N条经验出来
    def sample(self, batch_size):
        mini_batch = random.sample(self.buffer, batch_size)
        obs_batch, action_batch, reward_batch, next_obs_batch, done_batch = [], [], [], [], []

        for experience in mini_batch:
            s, a, r, s_p, done = experience
            obs_batch.append(s)
            action_batch.append(a)
            reward_batch.append(r)
            next_obs_batch.append(s_p)
            done_batch.append(done)

        return np.array(obs_batch).astype('float32'), \
            np.array(action_batch).astype('float32'), np.array(reward_batch).astype('float32'),\
            np.array(next_obs_batch).astype('float32'), np.array(done_batch).astype('float32')

    def __len__(self):
        return len(self.buffer)

In [30]:
# 训练一个episode
def run_episode(env, agent, rpm):
    total_reward = 0
    obs = env.reset()
    obs = obs.swapaxes(0, 2).swapaxes(1, 2)
    step = 0
    while True:
        step += 1
        action = agent.sample(obs)  # 采样动作，所有动作都有概率被尝试到
        next_obs, reward, done, _ = env.step(action)
        next_obs = next_obs.swapaxes(0, 2).swapaxes(1, 2)
        rpm.append((obs, action, reward, next_obs, done))

        # train model
        if (len(rpm) > MEMORY_WARMUP_SIZE) and (step % LEARN_FREQ == 0):
            (batch_obs, batch_action, batch_reward, batch_next_obs,
             batch_done) = rpm.sample(BATCH_SIZE)
            train_loss = agent.learn(batch_obs, batch_action, batch_reward,
                                     batch_next_obs,
                                     batch_done)  # s,a,r,s',done

        total_reward += reward
        obs = next_obs
        if done:
            break
    return total_reward


# 评估 agent, 跑 5 个episode，总reward求平均
def evaluate(env, agent, render=False):
    eval_reward = []
    #for i in range(5):
    obs = env.reset()
    episode_reward = 0
    while True:
        obs = obs.swapaxes(0, 2).swapaxes(1, 2)
        action = agent.predict(obs)  # 预测动作，只选最优动作
        obs, reward, done, _ = env.step(action)
        episode_reward += reward
        if render:
            env.render()
        if done:
            max_score = env.get_board().max()
            break
    #eval_reward.append(episode_reward)
    #return np.mean(episode_reward)
    return episode_reward,max_score

In [31]:
def create_env(board_size, seed, binary):
    env = gym.make("game2048-v0", board_size=board_size, seed=seed, binary=binary, extractor="cnn", penalty=-10)
    return env

In [32]:
# 创建环境
ACTION_LIST = ["↑", "↓", "→", "←"]
env = create_env(4,None,True)
act_dim = len(ACTION_LIST)  # 2048:4个动作(0:上, 1:下, 2:右, 3:左)
obs_dim = [16, 4, 4] 
logger.info('act_dim:{},obs_dim:{}'.format(act_dim,obs_dim[0]))
# 根据parl框架构建agent
model = Model(act_dim=act_dim)
algorithm = DQN(model, act_dim=act_dim, gamma=GAMMA, lr=LEARNING_RATE)
# 创建经验池
rpm = ReplayMemory(int(MEMORY_SIZE)) # DQN的经验回放池

[06-30 10:53:02 MainThread @<ipython-input-32-624029cd3a9f>:6] act_dim:4,obs_dim:16


In [29]:
agent = Agent(
    algorithm,
    obs_dim=obs_dim,
    act_dim=act_dim,
    e_greed=0.1, 
    e_greed_decrement=1e-6) 



# 加载模型
# save_path = './dqn_model.ckpt'
# agent.restore(save_path)

# 先往经验池里存一些数据，避免最开始训练的时候样本丰富度不够
while len(rpm) < MEMORY_WARMUP_SIZE:
    run_episode(env, agent, rpm)

# 开始训练
episode = 0
while episode < MAX_EPISODE:  # 训练max_episode个回合，test部分不计算入episode数量
    # train part
    for i in range(0, 50):
        total_reward = run_episode(env, agent, rpm)
        episode += 1
    # 训练结束，保存模型
    save_path = './work/dqn_model/dqn_model_{}.ckpt'.format(episode)
    agent.save(save_path)
    # test part
    eval_reward,max_score = evaluate(env, agent, render=False)  # render=True 查看显示效果
    logger.info('episode:{}    max_score:{}   test_reward:{}'.format(
        episode, max_score, eval_reward))

[06-30 10:53:04 MainThread @machine_info.py:86] nvidia-smi -L found gpu count: 1
[06-30 10:53:05 MainThread @machine_info.py:86] nvidia-smi -L found gpu count: 1
[06-30 10:53:06 MainThread @machine_info.py:86] nvidia-smi -L found gpu count: 1
[06-30 10:53:20 MainThread @<ipython-input-33-20ec996bda02>:31] episode:50    max_score:16   test_reward:-168
